# Chapter 8: Classes and Objects

The primary focus of this chapter is to present recipes to common programming patterns related to class definitions.  
Topics include making objects support common Python features, usage of special methods, encapsulation techniques, inheritance, memory management, and useful design patterns.  

## 8.1. Changing the String Representation of Instances

### Problem

You want to change the output produced by printing or viewing instances to something more sensible.

### Solution

To change the string representation of an instance, define the `__str__()` and `__repr__()` methods.

In [1]:
class Pair:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        # r for repr
        return "Pair({0.x!r}, {0.y!r})".format(self)
    def __str__(self):
        # s for str
        return "({0.x!s}, {0.y!s})".format(self)

The `__repr__()` method returns the code representation of an instance, and is usually the text you would type to recreate the instance.  
The built-in `repr()` function returns this text, as does the interactive interpreter when inspecting values.  
The `__str__()` method converts the instance to a string, and is the output produced by the `str()` and `print()` functions.

In [2]:
p = Pair(3, 4)
# __repr__() output:
p

Pair(3, 4)

In [3]:
#__str__() output:
print(p)

(3, 4)


The implementation of this recipe also shows how different string representations may be used during formatting.  
Specifically, the special `!r` formatting code indicates that the output of `__repr__()` should be used instead of the default`__str__()`.  
You can try this experiment with the preceding class to see this:  

In [4]:
p = Pair(3, 4)
print('p is {0!r}'.format(p))

p is Pair(3, 4)


In [5]:
print('p is {0}'.format(p))

p is (3, 4)


### Discussion

Defining `__repr__()` and `__str__()` is often good practice, as it can simplify debugging and instance output.  
For example, just by printing or logging an instance, a programmer will be shown more useful information about the instance contents.  
It is standard practice for the output of `__repr__()` to produce text such that `eval(repr(x)) == x`.  
If this is not possible or desired, then it is common to create a useful textual representation enclosed in `<` and `>` instead.

In [6]:
f = open('example.bin')
f

<_io.TextIOWrapper name='example.bin' mode='r' encoding='UTF-8'>

In [7]:
f.close()

If no `__str__()` is defined, the output of `__repr__()` is used as a fallback.  
The use of `format()` in the solution might look a little funny, but the format code `{0.x}` specifies the x-attribute of argument 0.  
So, in the following function, the 0 is actually the instance self:

In [8]:
def __repr__(self):
    return "Pair({0.x!r}, (0.y!r))".format(self)

You can also use the modulo operator:

In [9]:
def __repr__(self):
    return "Pair(%r, %r)" % (self.x, self.y)

In [10]:
p

Pair(3, 4)

In [11]:
print(p)

(3, 4)


## 8.2. Customizing String Formatting

### Problem

You want an object to support customized formatting through the `format()` function and string method.

### Solution

You can customize string formatting by defining the `__format__()` method on a class.

In [12]:
_formats = {
        'ymd' : '{d.year}-{d.month}-{d.day}',
        'mdy' : '{d.month}/{d.day}/{d.year}',
        'dmy' : '{d.day}/{d.month}/{d.year}'
        }

class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day
        
    def __format__(self, code):
        if code == '':
            code = 'ymd'
        fmt = _formats[code]
        return fmt.format(d=self)

Instances of the `Date` class now support operations like the ones below:

In [13]:
d = Date(2019, 3, 21)
format(d)

'2019-3-21'

In [14]:
format(d, 'mdy')

'3/21/2019'

In [15]:
"The date is {:ymd}".format(d)

'The date is 2019-3-21'

In [16]:
"The date is {:mdy}".format(d)

'The date is 3/21/2019'

### Discussion


The `__format__()` method provides a hook into Python’s string formatting functionality.  
It’s important to emphasize that the interpretation of format codes is entirely up to the class itself.  
Thus, the codes can be almost anything at all.  
For example, consider the following from the `datetime` module:

In [17]:
from datetime import date

d = date(2019, 3, 21)
format(d)

'2019-03-21'

In [18]:
format(d, '%A, %B, %d, %Y')

'Thursday, March, 21, 2019'

In [19]:
'The end is coming on {:%d %b %Y}. Farewell.'.format(d)

'The end is coming on 21 Mar 2019. Farewell.'

There are some standard conventions for the formatting of the built-in types.  
See the [documentation for the string module](https://docs.python.org/3/library/string.html) for a formal specification.

## 8.3. Making Objects Support the Context-Management Protocol

### Problem

You want to make your objects support the context-management protocol, aka the `with` statement.

### Solution

In order to make an object compatible with the `with` statement, you need to implement `__enter__()` and `__exit__()` methods.  
For example, consider the following class, which provides a network connection:

In [20]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family=AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.sock = None
        
    def __enter__(self):
        if self.sock is not None:
            raise RuntimeError('*** Already connected ***')
        self.sock = socket(self.family, self.type)
        self.sock.connect(self.address)
        return self.sock
    
    def __exit__(self, exc_ty, exc_val, tb):
        self.sock.close()
        self.sock = None

The key feature of this class is that it represents a network connection, but it doesn’t actually do anything initially (e.g., it doesn’t establish a connection).  
Instead, the connection is established and closed using the with statement (essentially on demand).

In [21]:
from functools import partial

conn = LazyConnection(('www.python.org', 80))
# Connection closed
with conn as s:
    # conn.__enter__() executes: connection open
    s.send(b'GET /index.html HTTP/1.0\r\n')
    s.send(b'Host: www.python.org\r\n')
    s.send(b'\r\n')
    resp = b''.join(iter(partial(s.recv, 8192), b''))
    # conn.__exit() executes: connection closed

### Discussion

The main principle behind writing a context manager is that you’re writing code that’s meant to surround a block of statements as defined by the use of the `with` statement.  
When the `with` statement is first encountered, the `__enter__()` method is triggered.  
The return value of `__enter__()` (if any) is placed into the variable indicated with the `as` qualifier.  
Afterward, the statements in the body of the with statement execute.  
Finally, the `__exit__()` method is triggered to clean up.  
This control flow happens regardless of what happens in the body of the with statement, including if there are exceptions.  
In fact, the three arguments to the `__exit__()` method contain the exception type, value, and traceback for pending exceptions (if any).  
The `__exit__()` method can choose to use the exception information in some way or to ignore it by doing nothing and returning `None` as a result.  
If `__exit__()` returns `True`, the exception is cleared as if nothing happened and the program continues executing statements immediately after the with block.  
One subtle aspect of this recipe is whether or not the `LazyConnection` class allows nested use of the connection with multiple with statements.  
As shown, only a single socket connection at a time is allowed, and an exception is raised if a repeated with statement is attempted when a socket is already in use.  
You can work around this limitation with a slightly different implementation, as shown here:

In [22]:
from socket import socket, AF_INET, SOCK_STREAM

class LazyConnection:
    def __init__(self, address, family = AF_INET, type=SOCK_STREAM):
        self.address = address
        self.family = AF_INET
        self.type = SOCK_STREAM
        self.connections = []
        
    def __enter__(self):
        sock = socket(self.family, self.type)
        sock.connect(self.address)
        self.connections.append(sock)
        return sock
    
    def __exit(self, exc_ty, exc_val, tb):
        self.connections.pop().close()

In this second version, the `LazyConnection` class serves as a kind of factory for connections.  
Internally, a list is used to keep a stack.  
Whenever `__enter__()` executes, it makes a new connection and adds it to the stack.  
The `__exit__()` method simply pops the last connection off the stack and closes it.  
It’s subtle, but this allows multiple connections to be created at once with nested with statements, as shown.  
Context managers are most commonly used in programs that need to manage resources such as files, network connections, and locks.  
A key part of such resources is they have to be explicitly closed or released to operate correctly.  
For instance, if you acquire a lock, then you have to make sure you release it, or else you risk deadlock.  
By implementing `__enter__()`, `__exit__()`, and using the with statement, it is much easier to avoid such problems, since the cleanup code in the `__exit__()` method is guaranteed to run no matter what.  
An alternative formulation of context managers is found in the `contextmanager` module in Recipe 9.22.  
A thread-safe version of this recipe can be found in Recipe 12.6.

## 8.4. Saving Memory When Creating a Large Number of Instances

### Problem

Your program creates a large number (e.g., millions) of instances and uses a large amount of memory.

### Solution

For classes that primarily serve as simple data structures, you can often greatly reduce the memory footprint of instances by adding the `__slots__` attribute to the class definition.

In [23]:
class Date:
    __slots__ = ['year', 'month', 'day']
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

When you define `__slots__`, Python uses a much more compact internal representation for instances.  
Instead of each instance consisting of a dictionary, instances are built around a small fixed-sized array, much like a tuple or list.  
Attribute names listed in the `__slots__` specifier are internally mapped to specific indices within this array.  
A side effect of using slots is that it is no longer possible to add new attributes to instances — you are restricted to only those attribute names listed in the `__slots__` specifier.

### Discussion

The memory saved by using `__slots__` varies according to the number and type of attributes stored.  
However, in general, the resulting memory use is comparable to that of storing data in a tuple.  
To give you an idea, storing a single `Date` instance without slots requires 428 bytes of memory on a 64-bit version of Python.  
If slots is defined, it drops to 156 bytes.  
In a program that manipulated a large number of dates all at once, this would make a significant reduction in overall memory use.  
Although slots may seem like a feature that could be generally useful, you should resist the urge to use it in most code.  
There are many parts of Python that rely on the normal dictionary-based implementation.  
In addition, classes that define slots don’t support certain features such as multiple inheritance.  
For the most part, you should only use slots on classes that are going to serve as frequently used data structures in your program (like if your program created millions of instances of a particular class).  
A common misperception of `__slots__` is that it is an encapsulation tool that prevents users from adding new attributes to instances.  
Although this is a side effect of using slots, this was never the original purpose.  
Instead, `__slots__` was always [intended to be an optimization tool](http://python-history.blogspot.com/2010/06/inside-story-on-new-style-classes.html).

## 8.5. Encapsulating Names in a Class

### Problem

You want to encapsulate restricted data on instances of a class, but you are concerned about Python's lack of access control.

### Solution

Rather than relying on language features to encapsulate data, Python programmers are expected to observe certain naming conventions concerning the intended usage of data and methods.  
The first convention is that any name that starts with a single leading underscore `(_)` should *always* be assumed to be internal implementation.

In [24]:
class A:
    def __init__(self):
        # internal attribute
        self._internal = 0
        # public attribute
        self.public = 1
        
    def public_method(self):
        """
        Any public method will do.
        """
        pass
    
    def _internal_method(self):
        """
        Any private method will do.
        """

Python doesn’t actually prevent someone from accessing internal names.  
However, doing so is considered impolite, and may result in fragile code.  
It should be noted, too, that the use of the leading underscore is also used for module names and module-level functions.  
For example, if you ever see a module name that starts with a leading underscore (like `_socket`), it’s an internal implementation.  
Likewise, module-level functions such as `sys._getframe()` should only be used with great caution.  
You may also encounter the use of two leading underscores `(__)` on names within class definitions.

In [25]:
class B: 
    def __init__(self):
        self.__private = 0
        
    def __private_method(self):
        """
        Did you know that a docstring alone can form a complete function in Python?
        You don't even need the pass keyword.
        Thanks Guido;)
        """
    def public_method(self):
        self.__private_method()
        pass

The use of double leading underscores causes the name to be mangled to something else.  
Specifically, the private attributes in the preceding class get renamed to `_B__private` and `_B__private_method`, respectively.  
At this point, you might ask what purpose such name mangling serves.  
The answer is inheritance — such attributes cannot be overridden via inheritance.

In [26]:
class C(B):
    def __init__(self):
        super().__init__()
        self.__private = 1 # Doesn't override B.__private
    def __private_method(self):
        """
        Doesn't override B.__private_method()
        """

Here, the private names `__private` and `__private_method()` get renamed to `_C__private` and `_C__private_method`, which are different than the mangled names in the base class `B`.

### Discussion

The fact that there are two different conventions (single underscore versus double underscore) for "private" attributes leads to the obvious question of which style you should use.  
For most code, you should probably just make your nonpublic names start with a single underscore.  
If, however, you know that your code will involve subclassing, and there are internal attributes that should be hidden from subclasses, use the double underscore instead.  
It should also be noted that sometimes you may want to define a variable that clashes with the name of a reserved word.  
For this, you should use a single trailing underscore.

In [27]:
lambda_ = 2.0
# so you don't clash with the lambda keyword

The reason for not using a leading underscore here is that it avoids confusion about the intended usage.  
In other words, the use of a leading underscore could be interpreted as a way to avoid a name collision rather than as an indication that the value is private.  
Using a single trailing underscore solves this problem.

## 8.6. Creating Managed Attributes

### Problem

You want to add extra processing, such as type checking or validation, to the getting or setting of an instance attribute.

### Solution

A simple way to customize access to an attribute is to define it as a [property](https://www.programiz.com/python-programming/property).  
For example, this code defines a `property` that adds simple type checking to an attribute:

In [28]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name
        
    # Getter function
    @property
    def first_name(self):
        return self._first_name
    
    # Setter function
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
            
    # Optional Deleter function
    @first_name.deleter
    def first_name(self):
        raise AttributeError('This attribute cannot be deleted')

In the preceding code, there are three related methods, all of which must have the same name.  
The first method is a getter function, and establishes `first_name` as being a property.  
The other two methods attach optional setter and deleter functions to the `first_name` property.  
It’s important to stress that the `@first_name.setter` and `@first_name.deleter` decorators won’t be defined unless `first_name` was already established as a property using the `@property` decorator.
A critical feature of a property is that it looks like a normal attribute, but access *automatically* triggers the getter, setter, and deleter methods.

In [29]:
a = Person('Guido')
a.first_name

'Guido'

When implementing a property, the underlying data (if any) still needs to be stored somewhere.  
Thus, in the getter and setter methods, you see direct manipulation of a `_first_name` attribute, which is where the actual data lives.  
In addition, you may ask why the `__init__()` method sets `self.first_name` instead of `self._first_name`.  
In this example, the entire point of the property is to apply type checking when setting an attribute.  
Thus, chances are you would also want such checking to take place during initialization.  
By setting `self.first_name`, the set operation uses the setter method instead of bypassing it by accessing `self._first_name`.  
Properties can also be defined for existing get and set methods.

In [30]:
class Person:
    def __init__(self, first_name):
        self.set_first_name(first_name)
        
    # Getter function
    def get_first_name(self):
        return self._first_name
    
    # Setter function
    def set_first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
        
    # Optional Deleter function
    def del_first_name(self):
        raise AttributeError("This attribute cannot be deleted")
        
    # Make a property from the existing get and set methods
    name = property(get_first_name, set_first_name, del_first_name)

### Discussion

A property attribute is actually a collection of methods bundled together.  
If you inspect a class with a property, you can find the raw methods in the `fget`, `fset`, and `fdel` attributes of the property itself.

Normally, you wouldn't call `fget` or `fset` directly, but they are triggered automatically when the property is accessed.  
Properties should only be used in cases where you actually need to perform extra processing on attribute access.  
Sometimes programmers coming from languages such as Java feel that all access should be handled by getters and setters, and that they should write code like this:

In [31]:
class Person:
    def __init__(self, first_name):
        self.first_name = first_name
    @property
    def first_name(self):
        return self._first_name
    @first_name.setter
    def first_name(self, value):
        self._first_name = value

Don’t write properties that don’t actually add anything extra like this.  
For one, it makes your code more verbose and confusing to others.  
Second, it will make your program run a lot slower.  
Lastly, it offers no real design benefit.  
Specifically, if you later decide that extra processing needs to be added to the handling of an ordinary attribute, you could promote it to a property without changing existing code.  
This is because the syntax of code that accessed the attribute would remain unchanged.

Properties can also be a way to define computed attributes.  
These are attributes that are not actually stored, but computed on demand.

In [32]:
import math

class Circle:
    def __init__(self, radius):
        self.radius = radius
    @property
    def area(self):
        return math.pi * self.radius ** 2
    @property
    def perimeter(self):
        return 2 * math.pi * self.radius

Here, the use of properties results in a very uniform instance interface in that `radius`, `area`, and `perimeter` are all accessed as simple attributes, as opposed to a mix of simple attributes and method calls.

In [33]:
c = Circle(4.0)
c.radius

4.0

In [34]:
# notice the lack of ()
c.area

50.26548245743669

In [35]:
# again no parentheses
c.perimeter

25.132741228718345

Although properties give you an elegant programming interface, sometimes you actually may want to directly use getter and setter functions.

This often arises in situations where Python code is being integrated into a larger infrastructure of systems or programs.  
For example, perhaps a Python class is going to be plugged into a large distributed system based on remote procedure calls or distributed objects.  
In such a setting, it may be much easier to work with an explicit get/set method (as a normal method call) rather than a property that implicitly makes such calls.

Last, please don’t write Python code that features a lot of repetitive property definitions.

In [36]:
class Person:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name
        
    @property
    def first_name(self):
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._first_name = value
        
    # Repeated property code, but for a different name, which is confusing and not DRY
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._last_name = value

Code repetition leads to bloated, error prone, and ugly code.  
As it turns out, there are much better ways to achieve the same thing using descriptors or closures.  
See Recipes 8.9 and 9.21.

## 8.7. Calling a Method on a Parent Class

### Problem 

You want to invoke a method in a parent class in place of a method that has been overridden in a subclass.

### Solution

To call a method in a parent (or superclass), use the `super()` function.

In [37]:
class A:
    def spam(self):
        print('A.spam')
        
class B(A):
    def spam(self):
        print('B.spam')
        # call parent spam function
        super().spam()

A very common use of `super()` is in the handling of the `__init__()` method to make sure that parents are properly initialized:

In [38]:
class A:
    def __init__(self):
        self.x = 0
        
class B(A):
    def __init__(self):
        super().__init__()
        self.y = 1

Another common use of `super()` is in code that overrrides any of Python's special methods.

In [39]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj
        
    # delegate attribute lookup to internal object
    def __getattr__(self, name):
        return getattr(self._obj, name)
    
    # delegate attribute assignment
    def __setattr__(self, name, value):
        if name.startswith('_'):
            # call original setattr
            super().__setattr__(name, value)
        else:
            setattr(self._obj, name, value)

In this code, the implementation of `__setattr__()` includes a name check.  
If the name starts with an underscore `(_)`, it invokes the original implementation of `__setattr__()` using `super()`.  
Otherwise, it delegates to the internally held object `self._obj`.  
It looks a little funny, but `super()` works even though there is no explicit base class listed.

### Discussion

Correct use of the `super()` function is actually one of the most poorly understood aspects of Python.  
Occasionally, you will see code written that directly calls a method in a parent like this:

In [40]:
class Base:
    def __init__(self):
        print('Base.__init__')
        
class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')

Although you can usually get away with this, it can lead to trouble with inheritance later on.

In [41]:
class Base:
    def __init__(self):
        print('Base.__init__')
        
class A(Base):
    def __init__(self):
        Base.__init__(self)
        print('A.__init__')
        
class B(Base):
    def __init__(self):
        Base.__init__(self)
        print('B.__init__')
        
class C(A,B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)
        print('C__init__')

Now watch what happens when the `Base.__init__()` method is invoked twice:

In [42]:
c = C()

Base.__init__
A.__init__
Base.__init__
B.__init__
C__init__


Maybe that double-invocation of `Base.__init__()` is harmless, but you won't have to worry if you write the code using `super()`.

In [43]:
class Base:
    def __init__(self):
        print('Base.__init__')
        
class A(Base):
    def __init__(self):
        super().__init__()
        print('A.__init__')
        
class B(Base):
    def __init__(self):
        super().__init__()
        print('B.__init__')
        
class C(A, B):
    def __init__(self):
        # single call to super
        super().__init__()
        print('C.__init__')

Now each `__init__` method is only called once:

In [44]:
c = C()

Base.__init__
B.__init__
A.__init__
C.__init__


To understand why it works, we need to step back for a minute and discuss how Python implements inheritance.  
For every class that you define, Python computes what’s known as a [method resolution order (MRO)](http://python-history.blogspot.com/2010/06/method-resolution-order.html) list.  
The MRO list is simply a linear ordering of all the base classes.  

In [45]:
print(C.__mro__)

(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.Base'>, <class 'object'>)


[First](https://en.wikipedia.org/wiki/C3_linearization#Example_demonstrated_in_Python_3), we have a metaclass to enable a short representation of the objects by name.

In [46]:
class Type(type):
    def __repr__(cls):
        return cls.__name__
    
class O(object, metaclass=Type): pass

Now construct the inheritance tree:

In [47]:
class A(O): pass

class B(O): pass

class C(O): pass

class D(O): pass

class E(O): pass

class K1(A, B, C): pass

class K2(D, B, E): pass

class K3(D, A): pass

class Z(K1, K2, K3): pass

In [48]:
print(Z.mro())

[Z, K1, K2, K3, D, A, B, C, E, O, <class 'object'>]


To implement inheritance, Python starts with the leftmost class and works its way left-to-right through classes on the MRO list until it finds the first attribute match.  
The actual determination of the MRO list itself is made using a technique known as [C3 Linearization](https://en.wikipedia.org/wiki/C3_linearization).  
Without getting too bogged down in the mathematics of it, it is actually a merge sort of the MROs from the parent classes subject to three constraints:  
* Child classes get checked before parents.  
* Multiple parents get checked in the order listed.  
* If there are two valid choices for the next class, pick the one from the first parent. 

Honestly, all you really need to know is that the order of classes in the MRO list accommodates almost any class hierarchy you are going to define.  
When you use the `super()` function, Python continues its search starting with the next class on the MRO.  
As long as every redefined method consistently uses `super()` and only calls it once, control will ultimately work its way through the entire MRO list and each method will only be called once.  
This is why you don’t get double calls to `Base.__init__()` in the second example.  
A somewhat surprising aspect of `super()` is that it doesn’t necessarily go to the direct parent of a class next in the MRO and that you can even use it in a class with no direct parent at all.  
For example, consider this class:

In [49]:
class A:
    def spam(self):
        print('A.spam')
        super().spam()

We have a problem:

However, watch what happens if you begin using the class with multiple inheritance:

In [50]:
class B:
    def spam(self):
        print('B.spam')
        
class C(A,B):
    pass

In [51]:
c = C()
c.spam()

A.spam
B.spam


Here you see that the use of `super().spam()` in class `A` has called the `spam()` method in class `B` -- a class that is completely unrelated to `A`.  
This is all explained by the MRO of class `C`:

In [52]:
print(C.__mro__)

(<class '__main__.C'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)


Using `super()` in this manner is most common when defining mixin classes.  
See Recipes 8.13 and 8.18.  
However, because `super()` might invoke a method that you’re not expecting, there are a few general rules of thumb you should try to follow.  
First, make sure that all methods with the same name in an inheritance hierarchy have a compatible calling signature, including the same number of arguments and argument names.  
This ensures that `super()` won’t get tripped up if it tries to invoke a method on a class that’s not a direct parent.  
Second, it’s usually a good idea to make sure that the topmost class provides an implementation of the method so that the chain of lookups that occur along the MRO get terminated by an actual method of some sort.  
Use of `super()` is sometimes a source of debate in the Python community.  
However, all things being equal, you should probably use it in modern code.  
Raymond Hettinger has written an excellent blog post [Python’s super() Considered Super!](https://rhettinger.wordpress.com/2011/05/26/super-considered-super/) that has even more examples and reasons why `super()` might be super-awesome.

## 8.8. Extending a Property in a Subclass

### Problem

Within a subclass, you want to extend the functionality of a property defined in a parent class.   

### Solution

Consider the following code, whch defines a property:

In [53]:
class Person:
    def __init__(self, name):
        self.name = name
        
    
    # getter function
    @property
    def name(self):
        return self._name
    
    # setter function
    @name.setter
    
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        self._name = value
        
    # deleter function
    @name.deleter
    def name(self):
        raise AttributeError("Can't delete attribute")

Here is an example of a class that inherits from `Person` and extends the `name` property with new functionality:

In [54]:
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name
    
    @name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)
        
    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)

Here is an example of our new class in use:

In [55]:
s = SubPerson('Guido')

Setting name to Guido


In [56]:
s.name

Getting name


'Guido'

In [57]:
s.name = 'Larry'

Setting name to Larry


If you only want to extend one of the methods of a property, use code such as the following:

In [58]:
class SubPerson(Person):
    @Person.name.getter
    def name(self):
        print('Getting name')
        return super().name

If you just want the setter, you can try this:

In [59]:
class SubPerson(Person):
    @Person.name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)

### Discussion

Extending a property in a subclass introduces a number of very subtle problems related to the fact that a property is defined as a collection of getter, setter, and deleter methods, as opposed to just a single method.  
Thus, when extending a property, you need to figure out if you will redefine all of the methods together or just one of the methods.  
In the first example, all of the property methods are redefined together.  
Within each method, `super()` is used to call the previous implementation.  
The use of `super(Sub Person, SubPerson).name.__set__(self, value)` in the setter function is no mistake.  
To delegate to the previous implementation of the setter, control needs to pass through the `__set__()` method of the previously defined name property.  
However, the only way to get to this method is to access it as a class variable instead of an instance variable.  
This is what happens with the `super(SubPerson, SubPerson)` operation.  
If you only want to redefine one of the methods, it’s not enough to use `@property` by itself.

In [60]:
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name

The following code will throw an `AttributeError` because the setter function can't be accessed.

With the code shown in the solution, however:

All of the previously defined methods of the property are copied, and the getter function is replaced.  
In this particular solution, there is no way to replace the hardcoded class name `Person` with something more generic.  
If you don’t know which base class defined a property, you should use the solution where all of the property methods are redefined and `super()` is used to pass control to the previous implementation.  
It’s worth noting that the first technique shown in this recipe can also be used to extend a descriptor, as described in Recipe 8.9.

In [61]:
# A descriptor
class String:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise TypeError('Expected a string')
        instance.__dict__[self.name] = value

        
# A class with a descriptor
class Person:
    name = String('name')
    def __init__(self, name):
        self.name = name

        
# Extending a descriptor with a property
class SubPerson(Person):
    @property
    def name(self):
        print('Getting name')
        return super().name
    
    @name.setter
    def name(self, value):
        print('Setting name to', value)
        super(SubPerson, SubPerson).name.__set__(self, value)
        
    @name.deleter
    def name(self):
        print('Deleting name')
        super(SubPerson, SubPerson).name.__delete__(self)

Finally, it’s worth noting that by the time you read this, subclassing of setter and deleter methods might be somewhat simplified.  
The solution shown will still work, but the [bug reported at Python’s issues page](https://bugs.python.org/issue14965) might evolve into a cleaner approach in a future Python version.

## 8.9. Creating a New Kind of Class or Instance Attribute

### Problem

You want to create a new kind of instance attribute type with some extra functionality, such as type checking.  

### Solution

If you want to create an entirely new kind of instance attribute, define its functionality in the form of a descriptor class. 

In [62]:
# Descriptor attribute for an integer type-checked attribute:
class Integer:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return instance.__dict__[self.name]
        
    def __set__(self, instance, value):
        if not isinstance(value, int):
            raise TypeError('Expected an integer')
        instance.__dict__[self.name] = value
        
    def __delete__(self, instance):
        del instance.__dict__[self.name]

A [descriptor](https://docs.python.org/3/howto/descriptor.html) is a class that implements the three core attribute access operations (get, set, and delete) in the form of `__get__()`, `__set__()`, and `__delete__()` special methods.  
These methods work by receiving an instance as input.  
The underlying dictionary of the instance is then manipulated as appropriate.  
To use a descriptor, instances of the descriptor are placed into a class definition as class variables.

In [63]:
class Point:
    x = Integer('x')
    y = Integer('y')
    def __init__(self, x, y):
        self.x = x
        self.y = y

When you do this, all access to the descriptor attributes x and y is captured by the `__get__()`, `__set__()`, and `__delete__()` methods.

In [64]:
p = Point(2, 3)
# calls Point.x.__get__(p, Point)
p.x

2

In [65]:
# calls Point.y.__set__(p, 5)
p.y = 5

As input, each method of a descriptor receives the instance being manipulated.  
To carry out the requested operation, the underlying instance dictionary (the `__dict__` attribute) is manipulated as appropriate.  
The `self.name` attribute of the descriptor holds the dictionary key being used to store the actual data in the instance dictionary.

### Discussion